<a href="https://colab.research.google.com/github/PraeJi/Handwritten_recognition/blob/main/handwritten_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gradio==3.35.0
! pip install torchvision

INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 

In [ ]:
import numpy as np
import torch
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import gradio as gr

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(),
    transforms.ToTensor()
])
labels = ["๐ (ศูนย์)", "๑ (หนึ่ง)", "๒ (สอง)", "๓ (สาม)", "๔ (สี่)", "๕ (ห้า)", "๖ (หก)", "๗ (เจ็ด)", "๘ (แปด)", "๙ (เก้า)"]
LABELS = {i:k for i, k in enumerate(labels)} # dictionary of index and label

# Load model
class DropoutThaiDigit(nn.Module):
  def __init__(self):
    super(DropoutThaiDigit, self).__init__()
    # Create layers
    self.fc1 = nn.Linear(28*28, 392)
    self.fc2 = nn.Linear(392, 196)
    self.fc3 = nn.Linear(196, 98)
    self.fc4 = nn.Linear(98, 10)
    self.dropout = nn.Dropout(p=0.1) # p คือ ค.น่าจะเป็นที่เราจะ drop activation layer ระหว่างเทรนโมเดล

  def forward(self, x):
    # forward pass
    x = x.view(-1, 28*28)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.fc4(x)
    return x

model = DropoutThaiDigit()
model.load_state_dict(torch.load("thai_digit_augmented.pth"))
model.eval()

/usr/local/lib/python3.10/dist-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/usr/local/lib/python3.10/dist-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
<ipython-input-2-97c93b956112>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only`

DropoutThaiDigit(
  (fc1): Linear(in_features=784, out_features=392, bias=True)
  (fc2): Linear(in_features=392, out_features=196, bias=True)
  (fc3): Linear(in_features=196, out_features=98, bias=True)
  (fc4): Linear(in_features=98, out_features=10, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
def predict(img):
  """
  Predict function takes image and return top 5 predictions as a dictionary:
  {label: confidence, label: confidence, ...}
  """
  if img is None:
    return None

  img = transform(img) # do not need to use 1-transform(img) because gradio already do it
  probs = model(img).softmax(dim=1).ravel()
  probs, indices = torch.topk(probs,5) # select top 5
  probs, indices = probs.tolist(), indices.tolist() # transform to list
  confidences = {LABELS[i]: v for i, v in zip(indices, probs)}

  return confidences

In [ ]:
gr.Interface(
    fn = predict,
    inputs = gr.Sketchpad(label="Draw Here", brush_redius=5, type="pil",shape=(120,120)),
    outputs = gr.Label(label="Guess"),
    title = "Thai Digit Handwritten Classification",
    live = True
).launch(enable_queue=True)

/usr/local/lib/python3.10/dist-packages/gradio/templates.py:120: UserWarning: You have unused kwarg parameters in Sketchpad, please remove them: {'brush_redius': 5}
  super().__init__(


IMPORTANT: You are using gradio version 3.35.0, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://decf445c1c99657cff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
